# Métodos de Computação Quântica com o Atos QLM

O ATOS Quantum Learning Machine (QLM) é um ambiente de teste para a computação quântica desenvolvida pela empresa francesa ATOS e adquirida pelo SENAI CIMATEC para o desenvolvimento de algoritmos quânticos no Centro de Computação Quântica da América Latina. O dispositivo é capaz de simular até 35 qubits e inclui a simulação de ruídos, emulando a arquitetura de computadores quânticos reais.

O QLM também possui interoperabilidade com outras frameworks de computação quântica, o que possibilita o uso de computadores quânticos através da plataforma. Além disso, o myQLM, framework disponibilizado gratuitamente, possibilita também o desenvolvimento e simulação de algoritmos quânticos por usuários domésticos, podendo sker instalado em seus computadores. Por sua completeza, a utilização do QLM para o desenvolvimento de algoritmos de computação quântica se torna interessante, podendo ser utilizado em diferentes etapas do desenvolvimento de algoritmos quânticos.

In [1]:
import uuid
import json
import shutil
from os import path, makedirs
from functools import partial
from datetime import datetime, timezone

import scipy
from pyqubo import Spin, Binary
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import qaoa

%matplotlib inline

# Otimização de Portfólio

A função objetivo para a otimização de portfólio pode ser modelada como uma função quadrática binária irrestrita. 

Resolver o seguinte problema de otimização: \begin{split}\begin{aligned} \min_{x \in \{0, 1\}^n} q x^T \sigma x - \mu^T x\\ \text{com a restrição: } 1^T x = B \end{aligned}\end{split}
onde:
- $x \in \{0,1\}^n$ denota o vetor de variáveis ​​de decisão binárias, que indicam quais ativos escolher ($x = 1$) e quais não escolher ($x =0$),
- $\mu \in \mathbf{R}^n$ define os retornos esperados para os ativos, 
- $\sigma \in \mathbf{R}^{n\times n}$ especifica a matriz de covariância entre os ativos,
- $q>0$ controla o apetite de risco do gestor do portfólio,
- e $B$ denota o orçamento, neste caso, o número de ativos a serem selecionados dentre $n$ disponíveis.

Observações:

1 - simplificações assumidas: i) todos os ativos tem o mesmo preço normalizado como $1$ (pensando no log dos preços, não parece uma simplificação tão absurda), ii) o orçamento total deve ser gasto, ou seja, é necessário selecionar exatamente $B$ ativos. 

2- $q$ pode ser visto como uma medida do apetite ao risco pois, ao aumentar $q$, precisamos de maiores retornos para o alcançar o mínimo da função objetiva, o que, pela Fronteira Eficiente, exige maiores riscos!

3- A restrição $1^T x = B$ pode ser imaginada como oriunda de um termo de penalidade do tipo $\alpha (1^T x - B)^2$ incluído explicitamente na função objetiva. Ou seja, o problema com a restrição acima é idêntico à otimização do seguinte:
\begin{split}\begin{aligned} \min_{x \in \{0, 1\}^n} q x^T \sigma x - \mu^T x + \alpha (1^T x - B)^2  \end{aligned}\end{split} pois considerando $\alpha >0$, a menor contribuição do termo de regularização ocorre quando $1^T x = B$.

4- O problema resultante pode ser mapeado para um hamiltoniano cujo estado fundamental corresponde à solução ótima.

Essa secção mostra como usar o Variational Quantum Eigensolver (VQE) e o Quantum Approximate Optimization Algorithm (QAOA) para encontrar a solução ideal para um determinado conjunto de parâmetros.

## 1. Quantum Approximate Optimization Algorithm (QAOA)

In [2]:
# matriz de covariante usada no problema clássico
cov = [[1.08774352e-03, 2.59532811e-04, 1.80247155e-04, 3.21724369e-04],
       [2.59532811e-04, 4.43192629e-04, 7.43211072e-05, 2.27911525e-04],
       [1.80247155e-04, 7.43211072e-05, 3.89444953e-04, 1.37915422e-04],
       [3.21724369e-04, 2.27911525e-04, 1.37915422e-04, 8.75437564e-04]]

# retornos
mu = [0.31542042, 0.0571331, 0.11430001, 0.30109367]

# parâmetros da função objetiva
q = 0.5
B = 2
lamb = 1

# cria as variáveis binárias
s1, s2, s3, s4 = Spin("s1"), Spin("s2"), Spin("s3"), Spin("s4")
x1, x2, x3, x4 = Binary("x1"), Binary("x2"), Binary("x3"), Binary("x4")

X = [x1, x2, x3, x4]

# escreve o QUBO
EN = (mu[0]*X[0] + mu[1]*X[1] + mu[2]*X[2] + mu[3]*X[3] -
      q*(cov[0][1]*X[0]*X[1] +
         cov[0][2]*X[0]*X[2] +
         cov[0][3]*X[0]*X[3] +
         cov[1][2]*X[1]*X[2] +
         cov[1][3]*X[1]*X[3] +
         cov[2][3]*X[2]*X[3]) -
      lamb*(B - X[0] - X[1] - X[2] - X[3]))

# cria o modelo e transforma de QUBO para Ising
model = EN.compile()
linear, quadratic, offset = model.to_ising()

# coeficientes que serão usados no Operador Gamma
coefs = [
    linear['x1'],
    linear['x2'],
    linear['x3'],
    linear['x4'],
    quadratic[('x1', 'x2')],
    quadratic[('x1', 'x3')],
    quadratic[('x1', 'x4')],
    quadratic[('x2', 'x3')],
    quadratic[('x2', 'x4')],
    quadratic[('x3', 'x4')]
]

In [3]:
num_assets = 4

num_qubits = num_assets

var_circuit = partial(
    qaoa.var_circuit,
    num_qubits=num_qubits,
    coefs=coefs
)

energy = partial(
    qaoa.energy,
    cov=cov,
    mu=mu,
    q=q,
    B=B,
    lamb=lamb,
    num_qubits=num_qubits
)


def make_cost_function(num_layers, num_shots):
    return qaoa.CostFunction(
        partial(var_circuit, num_layers=num_layers),
        energy,
        num_shots
    )

In [4]:
execution_json_path = "data/execution.json"
excel_file_path = 'data/qaoa_{num_layers}.xlsx'
makedirs('data', exist_ok=True)

In [5]:
def run_qaoa(num_layers: int, seed: int, num_shots) -> pd.DataFrame:
    assert (num_layers > 0)

    cost = make_cost_function(num_layers, num_shots)

    np.random.seed(seed)
    guess = np.random.random(2*num_layers)

    ret = scipy.optimize.minimize(
        cost,
        guess,
        args=(),
        method='COBYLA',
        constraints=(),
        tol=None,
        callback=None,
        options={
            'rhobeg': 1.0,
            'maxiter': 1000,
            'disp': False,
            'catol': 0.0002
        }
    )

    cost.save_executions(execution_json_path)

    p1001 = cost.get_last_execution()[9] / \
        num_shots if 9 in cost.get_last_execution() else 0.0

    data = {
        'ret': [ret['fun']],
        'p1001': [p1001],
        'nit': [ret['nfev']],
        'layers': [num_layers],
        'seed': [seed],
        'shots': [num_shots],
        'param': [list(ret['x'])],
        'id': [cost.id],
        'time': [str(datetime.now(timezone.utc))],
    }

    for key, value in data.items():
        value = value[0]
        if isinstance(value, float):
            value = f'{value:.3f}'
        print(f' {key}: {value}', end='\t|')
    print()

    return pd.DataFrame(data)

In [6]:

def run_qaoa_and_save(num_layers: int, seed: list[int], num_shots):
    excel_file = excel_file_path.format(num_layers=num_layers)

    try:
        df = pd.read_excel(excel_file)
    except FileNotFoundError:
        df = pd.DataFrame({
            'ret': [],
            'p1001': [],
            'nit': [],
            'layers': [],
            'seed': [],
            'shots': [],
            'param': [],
            'id': [],
            'time': [],
        })

    for s in seed:
        df = pd.concat(
            (df, run_qaoa(num_layers, s, num_shots)),
            ignore_index=True
        )
        df.to_excel(excel_file, index=False)

    print(
        f'\t->\t{num_layers=}\tret={df["ret"].mean():.3f}\tp1001={df["p1001"].mean():.3f}\tnit={df["nit"].mean():.3f}\n'
    )

In [7]:
for i in range(1, 21):
    run_qaoa_and_save(i, seed=[1 << j for j in range(5)], num_shots=1 << 20)

 ret: 0.404	| p1001: 0.030	| nit: 47	| layers: 1	| seed: 1	| shots: 1048576	| param: [0.6594953408354741, 0.2705110002717339]	| id: 033cd2cfd22d4da3a7cbdea481358bc0	| time: 2023-06-30 14:52:37.995191+00:00	|
 ret: 0.385	| p1001: 0.029	| nit: 29	| layers: 1	| seed: 2	| shots: 1048576	| param: [0.30137733909106973, 0.25113233260522294]	| id: 33af657b347a4751977f04d5a4d3376c	| time: 2023-06-30 14:52:39.651774+00:00	|
 ret: 0.399	| p1001: 0.026	| nit: 40	| layers: 1	| seed: 4	| shots: 1048576	| param: [0.5569474265207294, 0.2979720821325548]	| id: 9e5da89d584a4e95a96740588f19df0b	| time: 2023-06-30 14:52:41.668708+00:00	|
 ret: 0.302	| p1001: 0.059	| nit: 38	| layers: 1	| seed: 8	| shots: 1048576	| param: [2.9771686759862157, 2.084910670713899]	| id: 11388bb1440f4c97a2c2b0e40cd415b3	| time: 2023-06-30 14:52:43.667105+00:00	|
 ret: 0.419	| p1001: 0.028	| nit: 34	| layers: 1	| seed: 16	| shots: 1048576	| param: [0.8436276832371833, 0.3224842828089896]	| id: f4364a67f6f54521a0bd6b915c423472	|

# Notas sobre o QAOA

O QAOA é um algoritmo variacional da computação quântica, desenvolvido por Faihi em 2014, adequado para resolver problemas de otimização combinatória. Nesta seção, apresentaremos um problema de otimização combinatória que pode ser modelado como um modelo ferromagnético popular na mecânica estatística: o \textit{Modelo de Ising} (ou \textit{Hamiltoniano de Ising}). O algoritmo consiste em modelar a função objetivo (F.O.) como um \textit{Hamiltoniano de Ising} e transformá-la em operadores unitários, aplicar tais operadores em um estado de entrada, medir os valores esperados, atualizar os parâmetros e repetir o processo até que a função custo seja minimizada. Quando a energia da F.O. é minimizada, a distribuição de probabilidades (obtida após a medição da função de onda $|\psi\rangle$ com os parâmetros ótimos) apresenta uma ocorrência maior do autoestado de $|\psi\rangle$ que fornece a melhor configuração para o problema. 

### Modelo de Ising

O modelo do comportamento de materiais ferromagnéticos pode ser descrito pelo \textit{Hamiltoniano de Ising}. O modelo representa uma cadeia de partículas de spin 1/2, que interagem em pares, sendo que cada constituinte está sujeito à um campo magnético de magnitude $h_i$. Como partículas de spin 1/2 podem assumir apenas duas direções de momento de dipolo ("up" e "down") a interação entre cada par de partículas fornece uma energia (positiva ou negativa) de acordo com as orientações dos spins. Para encontrar a configuração dos momentos de dipolo que fornece o menor valor de energia possível do sistema, a seguinte função objetivo deve ser minimizada: 

\begin{equation} 
    C(z) = - \sum_{<ij>}^{}z_iz_j - \sum_{i}^{}h_iz_i .
\end{equation} 

As variáveis $z_i$ da função objetivo podem assumir apenas valores unitários, sejam eles positivos ou negativos, ou seja, $z_i \in \{-1,+1\}$. As variáveis podem ser escritas como operadores de Pauli-Z ($\sigma_z$), já que $\pm 1$ são autovalores de $\sigma_z$, associados às autofunções de spin das partículas.

### Operador de Fase

Para que seja possível mapear um problema de variáveis binárias $z_i$ para um problema de otimização de autovalores, uma estratégia é substituir as variáveis $z_i \in \{-1,+1\}$ por operadores unitários (que possuem autovalores $\lambda \in \{-1,+1\}$). Diante disso, é possível criar um Operador de Fase que representa as interações entre as partículas adjacentes, bem como a resposta das partículas ao campo magnético aplicado. Trata-se de escrever a função objetivo do problema como um operador unitário para que possa ser aplicada diretamente no circuito. Como visto, o operador descrito na Equação \ref{fo_ising} é não-unitário, visto que não satisfaz o critério da Equação 7. Contudo, cada termo da soma (negligenciando os coeficientes) é um operador unitário. Para descrever este Hamiltoniano define-se o Operador Gamma através da relação


\begin{equation} 
    U(C,\gamma)=e^{-i\gamma C(Z)}=e^{i\gamma\sum_{<ij>}^{}Z_iZ_j + i\gamma\sum_{i}^{}h_iZ_i} .
\end{equation}


Como $C(z)$ é expressa em termos dos operadores Pauli-Z e o produto tensorial entre eles geram matrizes diagonais, $U(C,\gamma)$ também é uma matriz diagonal (porém unitária), onde o parâmetro $\gamma$ é um dos parâmetros variacionais usados no QAOA. Desse modo, o operador $U(C,\gamma)$ pode ser escrito como a operação unitária

\begin{equation} 
    U(C,\gamma)=\prod_{<ij>}^{}e^{i\gamma Z_iZ_j}\prod_{i=0}^{N}e^{i\gamma h_iZ_i} .
\end{equation}

Esse operador (assim como produtos tensoriais entre matrizes de Pauli-Z) apresenta a forma de  uma matriz diagonal, onde os elementos da diagonal são exponenciais complexas que adicionam fases no estado quântico no qual é aplicado.

### Operador de Mistura

O operador $U(B,\beta)$ é aplicado com o objetivo de misturar as amplitudes (gerando interferências destrutivas e construtivas) possibilitando explorar melhor o espaço de busca e aumentando as chances de encontrar, com maior precisão, o ponto ótimo global da função objetivo. O Operador de Mistura pode ser descrito, matematicamente, como


\begin{equation} 
    U(B,\beta) = e^{-i\beta \sum_{j=0}^{N} X_j} \equiv \prod_{j=0}^{N}e^{-i\beta X_j}.
\end{equation}


Tal operador pode ser construído utilizando apenas operações de rotação em \textit{qubits} individuais do tipo RX. Contudo,a eficiência desse operador está diretamente ligada ao tempo de convergência e à acurácia do algoritmo. A Equação \ref{mix} descreve o operador em sua forma mais simples, representado apenas como rotações em torno do eixo $x$ da \textit{Esfera de Bloch}. 

### Função objetivo do QAOA: Medindo Valores Esperados
As operações unitárias apresentadas nas subseções anteriores podem ser representadas como diagramas de circuitos quânticos. No QAOA, cada conjunto das operações em questão são conhecidas como \textit{layers}. A $i$-ésima \textit{layer} do circuito pode ser escrita como


\begin{equation} 
    U(\gamma_i,\beta_i) = U(B,\beta_i)U(C,\gamma_i),
\end{equation}


e, portanto, a sequência desses operadores representa o operador geral do QAOA. Quanto maior o número de \textit{layers}, maior o número de parâmetros do circuito e, consequentemente, maior a acurácia do resultado. Assim como no VQE, a função objetivo do QAOA também é um valor esperado. A diferença entre eles é que, no VQE, utiliza-se um ansatz genérico e no QAOA, por sua vez, o ansatz é construído à partir da função objetivo. A atuação do operador geral do QAOA sobre o estado de entrada $|s\rangle$ pode ser considerada da seguinte maneira:
\begin{equation} 
    |\vec{\gamma},\vec{\beta} \rangle = (U(B,\beta_n)U(C,\gamma_n))...(U(B, \beta_1)U(C,\gamma_1))(U(B,\beta_0)U(C,\gamma_0))|s\rangle.
\end{equation}
Como é de interesse encontrar a configuração de menor energia para o hamiltoniano, a função objetivo do problema é, novamente, a função dos valores esperados de energia, que se minimizada, retorna o menor autovalor de energia do sistema. A função objetivo do problema pode ser expressa como

\begin{equation}
    \langle C(Z) \rangle \equiv \langle\vec{\gamma},\vec{\beta}|C(Z)|\vec{\gamma},\vec{\beta} \rangle.
\end{equation}
Como visto nos capitulos anteriores, o valor esperado de energia também pode ser escrito como um valor médio. Quando o estado quântico é preparado e medido uma quantidade suficiente de vezes, a distribuição de probabilidades obtida nos possibilita calcular o valor esperado como a soma da quantidade de vezes que cada autoestado da função de onda foi medido, multiplicada por suas respectivas energias associadas. Portanto, a função objetivo também pode ser escrita como
\begin{equation} 
    \langle\vec{\gamma},\vec{\beta}|C(Z)|\vec{\gamma},\vec{\beta} \rangle = E_0\alpha_0\langle\vec{\gamma},\vec{\beta}|0\rangle + E_1\alpha_1\langle\vec{\gamma},\vec{\beta}|1\rangle + E_2\alpha_2\langle\vec{\gamma},\vec{\beta}|2\rangle + ... + E_n\alpha_n\langle\vec{\gamma},\vec{\beta}|n\rangle =  \sum_{i=0}^{n}E_ip_i.
\end{equation}


Os parâmetros variacionais, $\gamma$ e $\beta$, são atualizados iterativamente através de métodos clássicos de otimização até que o menor autovalor de energia seja encontrado. Por se tratar de problemas de otimização combinatória, o interesse do problema é no autoestado da base computacional associado ao menor autovalor, que é composto pela \textit{bitstring} formada pela configuração ótima do problema. 


Como visto, o QAOA pode ser usado para encontrar a configuração que minimiza a energia do \textit{Hamiltoniano de Ising}. Por se tratar de um problema de otimização combinatória, percebe-se que o algoritmo é um bom candidato a resolver problemas dessa natureza. 

### O Método Variacional da Mecânica Quântica
Como discutido no Capítulo 2, informações sobre um estado quântico podem ser obtidas por meio de um conjunto de medidas sobre o sistema. Embora nosso objetivo seja encontrar o menor autovalor de uma matriz, a função objetivo do problema é calculada através valor esperado da matriz em questão, visto que se uma função de onda parametrizada por um conjunto de parâmetros $\theta$ é criada, o princípio variacional nos garante que
\begin{equation}
    \langle \mathcal{H}_\theta \rangle \equiv \langle \psi_\theta | \mathcal{H}|\psi_\theta \rangle \geq \lambda_{min}.
\end{equation}

Portanto, se o espaço de busca é suficientemente grande e o objetivo do problema é minimizar o valor esperado da matriz $ \mathcal{H}$, existe um conjunto de parâmetros $\theta'$ que minimiza a função objetivo. Diante disso, podemos afirmar que uma boa e confiável aproximação para o menor autovalor $\lambda{min}$ da matriz $\mathcal{H}$ pode ser dada por

\begin{equation}
    \langle \mathcal{H}_{\theta'}\rangle_{min} \approx \lambda_{min}.
\end{equation}

Uma explicação intuitiva do método variacional pode ser apresentada da seguinte maneira: O valor esperado (ou valor médio) de uma grandeza, pode assumir infinitos valores entre os respectivos máximo e mínimo. Logo, o valor médio de um observável físico nunca será menor que seu autovalor mínimo e nunca será maior que seu autovalor máximo. 
    
# Perspectivas
    
Futuros trabalhos nesta linha de pesquisa envolvem a inclusão de ruídos nas etapas quânticas da simulação. Esses ruídos ocorrem naturalmente nos computadores quânticos atuais e induzem erros que ainda não podem ser controlados eficientemente. Dessa forma, ao reproduzir os resultados com a inclusão de ruídos será possível determinar o ponto de transição onde os níveis de ruídos de um computador quântico passa a permitir a aplicação do  QAOA para a otimização de portfólios. 